In [24]:
from neo4j import GraphDatabase
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)

# Define Neo4j connection details
uri = "neo4j://localhost:7687"  # Adjust the host and port as needed

# Define your Cypher queries
cypher_queries = [
    # Test if wls was fully annotated to parents in hierarchy (should have 63 results from this query)
    ("MATCH (p:protein {name: 'wls', txid: 'txid7227'})-[:ProGo]-(g:go_term {id: 'GO:0030177'}) WITH p, g MATCH (p)-[:ProGo]-(g)-[:GoGo*]->(g2) RETURN count(g2)", 63),
    # Test that GO terms that are too specific are not annotated
    ("MATCH (p:protein {name: 'shf', txid: 'txid7227'})-[r:ProGo]-(g:go_term {id: 'GO:0016055'}) RETURN count(r)", 0),
    # Test that GO terms that have never_annotate are marked as never_annotate
    ("MATCH (g:go_term {id: 'GO:0009987'}) RETURN g.never_annotate", "true"),
    # Test that we did not overwrite direct annotations when propogating annotations
    ("MATCH (p:protein {name: 'dsh', txid: 'txid7227'})-[r:ProGo]-(g:go_term {id: 'GO:0016055'}) RETURN r.relationship", "involved_in")
]

# Function to run Cypher queries
def run_queries(driver, queries):
    overall_test_result = True  # Initialize overall test result
    with driver.session() as session:
        for query, expected_result in queries:
            try:
                result = session.run(query)
                record = result.single()
                actual_result = record[0] if record else None  # Extract result from record
                test_passed = (actual_result == expected_result)
                logging.info(f"Query: {query}, Expected: {expected_result}, Actual: {actual_result}, Test Passed: {test_passed}")
                if not test_passed:
                    overall_test_result = False
            except Exception as e:
                logging.error(f"Error executing query: {query}, Error: {str(e)}")
                test_passed = False
                overall_test_result = False
                continue  # Continue to the next query even if an error occurs
    return overall_test_result

# Connect to Neo4j
driver = GraphDatabase.driver(uri)

try:
    # Run Cypher queries and check results
    test_result = run_queries(driver, cypher_queries)
    print("All tests passed:", test_result)
finally:
    # Close the driver when done
    driver.close()

INFO:root:Query: MATCH (p:protein {name: 'wls', txid: 'txid7227'})-[:ProGo]-(g:go_term {id: 'GO:0030177'}) WITH p, g MATCH (p)-[:ProGo]-(g)-[:GoGo*]->(g2) RETURN count(g2), Expected: 63, Actual: 63, Test Passed: True


INFO:root:Query: MATCH (p:protein {name: 'shf', txid: 'txid7227'})-[r:ProGo]-(g:go_term {id: 'GO:0016055'}) RETURN count(r), Expected: 0, Actual: 0, Test Passed: True
INFO:root:Query: MATCH (g:go_term {id: 'GO:0009987'}) RETURN g.never_annotate, Expected: true, Actual: true, Test Passed: True
INFO:root:Query: MATCH (p:protein {name: 'dsh', txid: 'txid7227'})-[r:ProGo]-(g:go_term {id: 'GO:0016055'}) RETURN r.relationship, Expected: involved_in, Actual: involved_in, Test Passed: True


All tests passed: True
